In [ ]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error,mean_absolute_error,  mean_absolute_percentage_error

In [ ]:
!gdown #download link

In [ ]:
#Lets load the data
df = pd.read_excel('/content/Test Project Ercot.xlsx',sheet_name='DataSet')

In [ ]:
#Lets remove some redundant features
df.drop('MARKETDAY',axis=1,inplace=True) #Already captured
df.drop('MONTH',axis=1,inplace=True)     #Already captured
df.drop('PEAKTYPE',axis=1,inplace=True)  #Empty
df.drop('DATETIME',axis=1,inplace=True)  #Already captured
df.drop('YEAR',axis=1,inplace=True)      #No point taking this as a feature

df.dropna(axis=0,how='any',inplace=True) #Dropping rows with NA values

In [ ]:
y_real=df[['HB_NORTH (RTLMP)']]
y_ahead=df[['HB_NORTH (DALMP)']]
X=df.drop(['HB_HOUSTON (RTLMP)','HB_NORTH (RTLMP)','HB_PAN (RTLMP)','HB_SOUTH (RTLMP)','HB_WEST (RTLMP)',
           'HB_HOUSTON (DALMP)','HB_NORTH (DALMP)','HB_PAN (DALMP)','HB_SOUTH (DALMP)','HB_WEST (DALMP)'],axis=1) # 47+ 3 vars
lst = ['WZ_West (RTLOAD)','WZ_Southern (RTLOAD)','WZ_SouthCentral (RTLOAD)',
       'WZ_NorthCentral (RTLOAD)','WZ_North (RTLOAD)','WZ_Coast (RTLOAD)',
       'WZ_ERCOT (RTLOAD)','WZ_East (RTLOAD)','WZ_FarWest (RTLOAD)',
        'GR_COASTAL (WIND_RTI)','GR_ERCOT (WIND_RTI)','GR_NORTH (WIND_RTI)',
        'GR_PANHANDLE (WIND_RTI)','GR_SOUTH (WIND_RTI)','GR_WEST (WIND_RTI)',
        'ERCOT (GENERATION_SOLAR_RT)','ERCOT (AGG_DAM_ENERGY_SOLD)','ERCOT (AGG_DAM_ENERGY_BOUGHT)',
        'ERCOT (HSL_DA)','ERCOT (HSL_CD)','ERCOT (PHYS_RESP_CAP)','ERCOT (RT_OR_PRADDER)',
        'ERCOT (RT_ORD_PRADDER)']
X_future = X.drop(lst,axis=1).    #24 +3 vars
past_list = list(X_future.columns)
past_list.remove('day')
past_list.remove('month')
past_list.remove('HOURENDING')
X_past = X.drop(past_list,axis=1) #23 +3 vars

In [ ]:
X.values.shape

(4218, 50)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_y_DA,sc_y_real, sc_fut, sc_past, sc_all = StandardScaler(), StandardScaler(), StandardScaler(), StandardScaler(), StandardScaler()
X_future = pd.DataFrame(sc_fut.fit_transform(X_future),index=X_future.index,columns=X_future.columns)
X_past = pd.DataFrame(sc_past.fit_transform(X_past),index=X_past.index,columns=X_past.columns)
y_ahead = pd.DataFrame(sc_y_DA.fit_transform(y_ahead),index=y_ahead.index,columns=y_ahead.columns)
y_real = pd.DataFrame(sc_y_real.fit_transform(y_real),index=y_real.index,columns=y_real.columns)
X = pd.DataFrame(sc_all.fit_transform(X),index=X.index,columns=X.columns)

### NN and utils

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

dev=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
months=[10,11,12,1,2,3]
months_scaled = X_future['month'].unique().tolist()

months_name=['Oct','Nov','Dec','Jan','Feb','Mar']
target_months_name=['Jan','Feb','Mar']

In [ ]:
def evaluateNN(months,X_full, X, y, y_scaled=False,scaler=None,aug=False,X_original=None,y_original=None,do_return=False,backtest=True,unfreeze=False):
    if y_scaled:
        assert scaler is not None
    if aug:
        assert X_original is not None
        assert y_original is not None
        
    MSE_train=[]
    MSE_test=[]
    MAE_train=[]
    MAE_test=[]

    MAE_pretrain_train=[]
    MSE_pretrain_train=[]
    
    target_months = months[3:]

    if backtest:
        y_pred_final=[]
        for i,month in enumerate(target_months):
            network = MyDNN(50,27)
            network=network.to(dev)
            trainer = MyDNNTrain(network)

            ind_test = X.index[(X['month']==month)].tolist()
            ind_train = X.index[(X['month']<month) | (X['month']>=0)].tolist()
        
            X_train, X_test, y_train, y_test = np.array(X.loc[ind_train]), np.array(X.loc[ind_test]), np.array(y.loc[ind_train]), np.array(y.loc[ind_test])

            X_full_train = np.array(X_full.loc[ind_train])

            if aug:
                ind_original_test = X_original.index[(X_original['month']==month)].tolist()
                X_test = np.array(X_original.loc[ind_original_test])
                y_test = np.array(y_original.loc[ind_original_test])

            #pretraining
            network.pretrain = True
            trainer.train(y_train,X_full_train)

            with torch.no_grad():
                y_pred_train = network.predict(X_full_train)
                if y_scaled:
                    y_pred_train = scaler.inverse_transform(y_pred_train)

                MSE_pretrain_train.append(round(mean_squared_error(y_pred_train,scaler.inverse_transform(y_train)),2))
                MAE_pretrain_train.append(round(mean_absolute_error(y_pred_train,scaler.inverse_transform(y_train)),2))
            
            #finetuning
            if unfreeze is True:
                network.unfreeze=True
            network.pretrain=False
            trainer.train(y_train,X_train)
                
            with torch.no_grad():
                y_pred_train= network.predict(X_train)
                y_pred = network.predict(X_test)
                    
                    
                if y_scaled:
                    y_train=scaler.inverse_transform(y_train)
                    y_test = scaler.inverse_transform(y_test)
                    y_pred_train = scaler.inverse_transform(y_pred_train)
                    y_pred = scaler.inverse_transform(y_pred)
                        
                        
                MSE_train.append(round(mean_squared_error(y_train,y_pred_train),2))
                MAE_train.append(round(mean_absolute_error(y_train,y_pred_train),2))
                MSE_test.append(round(mean_squared_error(y_test,y_pred),2))
                MAE_test.append(round(mean_absolute_error(y_test,y_pred),2))
            
            y_pred_final.append(y_pred)
        
    # plt.plot(months_name,MSE_train,label='MSE train')
    # plt.plot(months_name,MAE_train,label='MAE train')
    # plt.plot(months_name,MSE_test,label='MSE test')
    # plt.plot(months_name,MAE_test,label='MAE test')
    # plt.legend()
    # plt.show()
    print('MSE_train FT',MSE_train, 'avg:', round(sum(MSE_train)/len(MSE_train),2))
    print('MSE_test FT',MSE_test,'avg:', round(sum(MSE_test)/len(MSE_train),2))
    print('MAE_train FT',MAE_train,'avg:', round(sum(MAE_train)/len(MSE_train),2))
    print('MAE_test FT',MAE_test,'avg:', round(sum(MAE_test)/len(MSE_train),2))
    print('MSE_pretrain_train PT',MSE_pretrain_train, 'avg:', round(sum(MSE_pretrain_train)/len(MSE_pretrain_train),2))
    print('MAE_pretrain_train PT',MAE_pretrain_train, 'avg:', round(sum(MAE_pretrain_train)/len(MAE_pretrain_train),2))

    if do_return:
        if backtest is False:
            return y_pred
        else:
            return np.concatenate(y_pred_final,axis=0)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, labels, features):
        super(MyDataset, self).__init__()
        self.labels = labels
        self.features = features

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]
        return {'feature': feature, 'label': label}

class MyDNN(nn.Module):
    def __init__(self, pretrain_inp, train_inp, pretrain=False):
        super(MyDNN, self).__init__()

        self.fc_m1 = nn.Linear(pretrain_inp,100)
        self.fc_m2 = nn.Linear(100, 50)
        self.fc_m3 = nn.Linear(50, 25)
        self.fc_m4 = nn.Linear(25, 1)


        self.fc_p1 = nn.Linear(train_inp,70)
        self.fc_p2 = nn.Linear(70,40)
        self.fc_p3 = nn.Linear(40,23)

        self.pretrain = pretrain
        self.unfreeze = False
    
    def switch(self,signal):
        self.fc_m1.weight.requires_grad = signal
        self.fc_m2.weight.requires_grad = signal
        self.fc_m3.weight.requires_grad = signal
        self.fc_m4.weight.requires_grad = signal
        self.fc_m1.bias.requires_grad = signal
        self.fc_m2.bias.requires_grad = signal
        self.fc_m3.bias.requires_grad = signal
        self.fc_m4.bias.requires_grad = signal


    def forward(self, x):
        if self.pretrain:
            self.switch(True)
            x = F.relu(self.fc_m1(x))
            x = F.relu(self.fc_m2(x))
            x = F.relu(self.fc_m3(x))
            x = self.fc_m4(x)
            return x
        else:
            self.switch(self.unfreeze)
            x1 = F.relu(self.fc_p1(x))
            x1 = F.relu(self.fc_p2(x1))
            x1 = self.fc_p3(x1)
            
            inp = torch.cat((x,x1),dim=1)

            inp = F.relu(self.fc_m1(inp))
            inp = F.relu(self.fc_m2(inp))
            inp = F.relu(self.fc_m3(inp))
            inp = self.fc_m4(inp)
            return inp

    def predict(self, features):
        self.eval()	
        features = torch.from_numpy(features).float().to(dev)
        return self.forward(features).detach().to('cpu').numpy()

class MyDNNTrain(object): 
    def __init__(self, network):
        self.network = network
        self.learning_rate = .0001
        self.optimizer = torch.optim.Adam(self.network.parameters(), lr=self.learning_rate,weight_decay=1e-6)
        self.criterion = nn.MSELoss()
        self.num_epochs = 200
        self.batchsize = 100
        self.shuffle = True
        self.dev=dev

    def train(self, labels, features):
        self.network.train()
        dataset = MyDataset(labels, features)
        loader = DataLoader(dataset, shuffle=self.shuffle, batch_size = self.batchsize)
        for epoch in range(self.num_epochs):
            self.train_epoch(loader, epoch)

    def train_epoch(self, loader, epoch):
        total_loss = 0.0
        for i, data in enumerate(loader):
            features = data['feature'].float().to(dev)
            labels = data['label'].float().to(dev)
            self.optimizer.zero_grad()
            predictions = self.network(features)
            loss = self.criterion(predictions, labels)
            loss.backward()
            total_loss += loss.item()
            self.optimizer.step()
        #print( 'Epoch',epoch,'loss', total_loss/i)

In [ ]:
Xnp = np.array(X)
n = Xnp.shape[0]
D=Xnp.shape[1]
X_aug=np.ones((1,D))
y_aug=np.ones(1)
indexes=[]

random.seed(0)
for i in range(10000):
    ind = random.randint(0,n)
    indexes.append(ind)
    y_aug=np.vstack((y_aug,y_real.iloc[ind]))
    x = Xnp[ind,:]
    x[:D-3]=x[:D-3]+np.random.normal(0,0.1,(D-3,))
    X_aug=np.vstack((X_aug,x))
X_aug=pd.DataFrame(X_aug[1:,:],columns=X.columns)
y_aug=pd.DataFrame(y_aug[1:,:],columns=y_real.columns)

X_new=pd.concat([X,X_aug])
y_new=pd.concat([y_real,y_aug])

X_new_future_only = X_new.drop(lst,axis=1)

In [ ]:
net=evaluateNN(months_scaled,X_new, X_new_future_only, y_new,y_scaled=True, scaler=sc_y_real, aug=True, X_original=X_future ,y_original=y_real)

MSE_train FT [3336.38, 3075.01, 7717.0] avg: 4709.46
MSE_test FT [624.65, 45145.49, 1107.73] avg: 15625.96
MAE_train FT [18.27, 17.83, 19.75] avg: 18.62
MAE_test FT [14.76, 36.96, 19.51] avg: 23.74
MSE_pretrain_train PT [34.79, 21.51, 28.94] avg: 28.41
MAE_pretrain_train PT [3.51, 2.91, 3.16] avg: 3.19


In [ ]:
net=evaluateNN(months_scaled,X_new, X_new_future_only, y_new,y_scaled=True, scaler=sc_y_real, aug=True, X_original=X_future ,y_original=y_real,unfreeze=True)
# WOW- Representation Learning actually works!

MSE_train FT [60.79, 36.79, 54.04] avg: 50.54
MSE_test FT [345.64, 3342.8, 175.16] avg: 1287.87
MAE_train FT [4.58, 3.86, 4.63] avg: 4.36
MAE_test FT [6.89, 9.1, 7.66] avg: 7.88
MSE_pretrain_train PT [30.16, 21.81, 23.5] avg: 25.16
MAE_pretrain_train PT [3.23, 2.93, 2.87] avg: 3.01
